In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune
import  sklearn.manifold as nonlin


In [56]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

mnist_train = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

trainloader_mnist =  torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader_mnist =  torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                          shuffle=True, num_workers=2)



                                        


Files already downloaded and verified
Files already downloaded and verified


In [57]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [36]:
path = './cifar_net.pt'
model = nn.Module()
model = torch.load(path)
model.eval()

NN(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [99]:
path_mnist = './custom_nn_mnist.pt'
model_mnist = torch.load(path_mnist)
model_mnist.eval()


NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [100]:
fc_1 = model_mnist.fc1.weight.detach().numpy().T
fa = nonlin.Isomap(n_components=60)
fc_1_reduced = fa.fit_transform(fc_1)
fc_1_reduced = torch.tensor(fc_1_reduced.T,  dtype=torch.float32)
print(fc_1_reduced.shape)


torch.Size([60, 256])


In [101]:
# fc_2 = nn.functional.relu(model_mnist.fc2.weight)
fc_2 = model_mnist.fc2.weight.detach().numpy()
fc_2 = fc_2.reshape(fc_2.shape[0]*2, -1)
fa_input = nonlin.Isomap(n_components=30)
fc_2_reduced =fa_input.fit_transform(fc_2)
print('one dimension reduced:')
fc_2_reduced = fc_2_reduced.reshape(-1, fc_2_reduced.shape[1] * 2)
print(fc_2_reduced.shape)
fc_2_reduced = fc_2_reduced.reshape(42, -1)
fa_ouput = nonlin.Isomap(n_components=21)
fc_2_reduced = fa_ouput.fit_transform(fc_2_reduced.T)
fc_2_reduced = fc_2_reduced.reshape(-1, fc_2_reduced.shape[1] * 2)
fc_2_reduced = fc_2_reduced.T
print('second dimension reduced:')

fc_2_reduced = torch.tensor(fc_2_reduced, dtype=torch.float32)
print(fc_2_reduced.shape)



one dimension reduced:
(84, 60)
second dimension reduced:
torch.Size([42, 60])


In [102]:
fc_3 =  model_mnist.fc3.weight.detach().numpy().T
fc3_Isomap = nonlin.Isomap(n_components=5, n_neighbors=5)
fc_3_reduced = fc3_Isomap.fit_transform(fc_3)
fc_3_reduced = fc_3_reduced.reshape(-1, 10)
fc_3_reduced = torch.tensor(fc_3_reduced.T, dtype=torch.float32)
print(fc_3_reduced.shape)


torch.Size([10, 42])


In [106]:
model_mnist.fc1 = nn.Linear(256, 60,bias=True)
model_mnist.fc2 = nn.Linear(60, 42,bias=True)
model_mnist.fc3 = nn.Linear(42, 10,bias=True)

model_mnist.fc1.weight = nn.Parameter(fc_1_reduced)
model_mnist.fc2.weight = nn.Parameter(fc_2_reduced)
model_mnist.fc3.weight = nn.Parameter(fc_3_reduced)


In [107]:
total = 0
acc=0
correct = 0
for data in testloader_mnist:
    pics, lables = data
    output = model_mnist(pics)
    _, pred = torch.max(output.data, 1)
    correct += (pred == lables).sum().item()
    total += lables.size(0)

print('average acc:', 100 * correct/ total)



average acc: 71.93


In [110]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim
running_loss = 0.0
total = 0.0
correct = 0.0
loss_nodr =[]
acc_nodr = []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_mnist.parameters(), lr=0.001, momentum=0.9)

print(optimizer)




for epoch in range(5):
    
    print('epoch:', epoch)
    for i, data in enumerate(trainloader_mnist):
        
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad(True)
        with torch.set_grad_enabled(True):
            with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
             outputs = model_mnist(inputs).to(device)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        if i % 2000 == 1999:
            loss_temp = running_loss / 2000
            acc_temp = 100 * correct / total
            loss_nodr.append(loss_temp)
            acc_nodr.append(acc_temp)
            print(
                f'[{epoch + 1}, {i + 1:5d}] loss: {loss_temp:.3f} acc: {acc_temp:.3f}')
            running_loss = 0.0
            correct = 0.0
            total = 0.0
print('Finished Training')


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 0.463 acc: 86.075
[1,  4000] loss: 0.242 acc: 93.037
[1,  6000] loss: 0.193 acc: 94.325
[1,  8000] loss: 0.170 acc: 94.850
[1, 10000] loss: 0.127 acc: 96.013
[1, 12000] loss: 0.132 acc: 95.950
[1, 14000] loss: 0.121 acc: 96.525
epoch: 1
[2,  2000] loss: 0.166 acc: 96.575
[2,  4000] loss: 0.099 acc: 96.925
[2,  6000] loss: 0.094 acc: 97.162
[2,  8000] loss: 0.099 acc: 97.175
[2, 10000] loss: 0.084 acc: 97.525
[2, 12000] loss: 0.088 acc: 97.375
[2, 14000] loss: 0.094 acc: 97.312
epoch: 2
[3,  2000] loss: 0.130 acc: 97.417
[3,  4000] loss: 0.082 acc: 97.600
[3,  6000] loss: 0.070 acc: 97.725
[3,  8000] loss: 0.076 acc: 97.662
[3, 10000] loss: 0.080 acc: 97.650
[3, 12000] loss: 0.065 acc: 98.100
[3, 14000] loss: 0.079 acc: 97.800
epoch: 3
[4,  2000] loss: 0.098 acc: 97.850
[4,  4000] loss: 0.066 acc: 98.025
[4,  6000] loss: 0.073 acc: 97.850
[4,  8000] loss: 0.063 acc: 98.237
[4, 10000] loss: 0.068 acc: 98.150
[4, 12000] loss: 0.064 acc: 98.062
[4, 14000] loss: 0.064 acc: 

In [71]:
path = './custom_nn_mnist.pt'
torch.save(model_mnist, path)

In [112]:
path ='./custom_nn_mnist_isomap.pt'
torch.save(model_mnist, path)